<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/240204-%E5%B9%B3%E5%9D%87HANTS%E4%B8%80%E9%98%B6RMSE%E3%80%81MAE%E3%80%81R2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-grn')

In [2]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# 筛选Sentinel-2影像

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2020-01-01','2024-01-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

2646


# HANTS

In [5]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [6]:
# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

# 评价拟合效果

In [9]:
points = [ee.Geometry.Point(103.69442853431933, 30.551562081391808),ee.Geometry.Point(103.65488522617737, 30.533297603476893),ee.Geometry.Point(103.59582889182555, 30.52539161730913),ee.Geometry.Point(103.56952465471126, 30.499742868049715),ee.Geometry.Point(103.53188984182017, 30.53036797408478),ee.Geometry.Point(103.50056911602078, 30.552690233958323),ee.Geometry.Point(103.46803551473484, 30.532530638095704),ee.Geometry.Point(103.49593611002624, 30.504634501905347),ee.Geometry.Point(103.53714266286114, 30.476484196954942),ee.Geometry.Point(103.56092304877916, 30.447647190442307),ee.Geometry.Point(103.59234187480173, 30.447031834496933),ee.Geometry.Point(103.62085336693748, 30.43962526764716),ee.Geometry.Point(103.59230620199332, 30.470990584457095),ee.Geometry.Point(103.6846051346986, 30.499778540858138),ee.Geometry.Point(103.75460410303558, 30.43591529557069),ee.Geometry.Point(103.84445053016147, 30.44333078062257),ee.Geometry.Point(103.85013142490355, 30.467811245405997),ee.Geometry.Point(103.6853230499682, 30.687827750484054),ee.Geometry.Point(103.65355195496241, 30.710867925627216),ee.Geometry.Point(103.62980724185279, 30.74277279366462),ee.Geometry.Point(103.59528488149701, 30.76877381190727),ee.Geometry.Point(103.5656229412895, 30.794079210385572),ee.Geometry.Point(103.72433126598374, 30.713917950747774),ee.Geometry.Point(103.7494003321062, 30.683622818190667),ee.Geometry.Point(103.7196982599892, 30.600558683767073),ee.Geometry.Point(103.74761669168483, 30.576216451116313),ee.Geometry.Point(103.74976597839259, 30.553020207436287),ee.Geometry.Point(103.77419739306443, 30.517467794737165),ee.Geometry.Point(103.8159524153289, 30.52715296222525),ee.Geometry.Point(103.81215326123136, 30.56217920100006),ee.Geometry.Point(103.81038745721419, 30.575338008208785),ee.Geometry.Point(103.77867878962313, 30.57299252105467),ee.Geometry.Point(103.91577831061238, 30.848248369982826),ee.Geometry.Point(103.85826928432608, 30.8489216942419),ee.Geometry.Point(103.85467078977598, 30.80402746483581),ee.Geometry.Point(103.79827653875303, 30.829444340840453),ee.Geometry.Point(103.74891874919244, 30.8550618763925),ee.Geometry.Point(103.79208284738979, 30.901851223746313),ee.Geometry.Point(103.82706895425513, 30.885299040635918),ee.Geometry.Point(103.85819793870925, 30.90270291204751),ee.Geometry.Point(103.82008154290438, 30.935258808838725),ee.Geometry.Point(103.7561647884043, 30.985124935919398),ee.Geometry.Point(103.75696742659392, 30.939191735967864),ee.Geometry.Point(103.70373021911682, 30.939361181807893),ee.Geometry.Point(103.88234843001472, 31.089717610229034),ee.Geometry.Point(103.95180338802317, 31.094002806341397),ee.Geometry.Point(103.95662367626194, 31.122732794428746),ee.Geometry.Point(104.00559798313195, 31.09046673920601),ee.Geometry.Point(104.03197802496413, 31.058004483536926),ee.Geometry.Point(103.98008746600517, 31.060274165973127),ee.Geometry.Point(103.98037730757366, 31.010274265860843),ee.Geometry.Point(104.03809591160946, 31.00245746171415),ee.Geometry.Point(104.07186914298828, 31.02673280784911),ee.Geometry.Point(104.10684187255043, 31.003884374051253),ee.Geometry.Point(104.10351538316458, 30.97206868803492),ee.Geometry.Point(104.1242011529515, 30.924481161592546),ee.Geometry.Point(104.07052695357117, 30.936052528826238),ee.Geometry.Point(104.04898949548304, 30.929769655441927),ee.Geometry.Point(104.03893868170857, 30.95408959258743),ee.Geometry.Point(104.03784174284941, 30.901320590720943),ee.Geometry.Point(104.07251125353997, 30.899523572996404),ee.Geometry.Point(104.12619883022344, 30.876840125937523),ee.Geometry.Point(104.16585807499278, 30.862031451339032),ee.Geometry.Point(104.19714312798378, 30.843468213653537),ee.Geometry.Point(104.20473697707779, 30.787947946437082),ee.Geometry.Point(104.2233091329654, 30.746875166633796),ee.Geometry.Point(104.3223769810698, 30.867208467662074),ee.Geometry.Point(104.34797668021764, 30.862905435145514),ee.Geometry.Point(104.32128004221065, 30.792732561867428),ee.Geometry.Point(103.69006307438801, 30.47595356392958),ee.Geometry.Point(103.68279919877195, 30.42133849422698),ee.Geometry.Point(103.65444377517308, 30.390432464825544),ee.Geometry.Point(103.62306954016104, 30.394566051502085),ee.Geometry.Point(103.43673262533966, 30.473460926440712),ee.Geometry.Point(103.56000447396217, 30.390294232692884),ee.Geometry.Point(103.67927204983867, 30.311903236173308),ee.Geometry.Point(103.76927900460247, 30.4178202634958),ee.Geometry.Point(103.68609447445044, 30.57786631850608),ee.Geometry.Point(103.58700878994183, 30.562544847286443),ee.Geometry.Point(103.62182991006817, 30.5831235986481),ee.Geometry.Point(103.61745107283369, 30.616794270702663),ee.Geometry.Point(103.56936412707333, 30.622822975326926),ee.Geometry.Point(103.59710419472684, 30.66326256278062),ee.Geometry.Point(103.63035125218131, 30.69151988615631),ee.Geometry.Point(103.62570040978258, 30.634795661655428),ee.Geometry.Point(103.6501318244544, 30.606685488614506),ee.Geometry.Point(103.66395503772009, 30.675248626412294),ee.Geometry.Point(103.81523004095824, 30.638768720694053),ee.Geometry.Point(103.87737653234012, 30.7408687575148),ee.Geometry.Point(103.90792137455621, 30.738166542276414),ee.Geometry.Point(103.8189712267421, 30.766883153060604),ee.Geometry.Point(103.63646913882664, 30.784483224918553),ee.Geometry.Point(103.62724771784812, 30.828280515465508),ee.Geometry.Point(103.62666803471116, 30.880425243184494),ee.Geometry.Point(103.63250053888908, 30.939352263605787),ee.Geometry.Point(103.67458107553044, 31.064180338495948),ee.Geometry.Point(103.70539346380976, 31.035976524332895),ee.Geometry.Point(104.01139927360248, 30.97991224678793),ee.Geometry.Point(103.98224121181401, 30.95262254834084),ee.Geometry.Point(103.9124830349339, 30.953741782705258),ee.Geometry.Point(103.85152712353329, 30.95874043498617),ee.Geometry.Point(103.79579727856733, 30.96167452347934),ee.Geometry.Point(103.78737403667736, 31.040792353470632),ee.Geometry.Point(103.85454147584541, 31.06187052415026),ee.Geometry.Point(103.88376196504863, 31.045844514964173),ee.Geometry.Point(103.84999319277091, 31.0103456114777),ee.Geometry.Point(103.91530118679968, 31.007634478037204),ee.Geometry.Point(103.97613224337078, 30.898939430758407),ee.Geometry.Point(103.94322407759635, 30.873669705088535),ee.Geometry.Point(103.97305992274497, 30.848172565264928),ee.Geometry.Point(103.99353165568132, 30.86875577572763),ee.Geometry.Point(104.04493617262546, 30.849260585921964),ee.Geometry.Point(104.10052778545875, 30.837296817795565),ee.Geometry.Point(104.12129381906466, 30.817476113613),ee.Geometry.Point(103.84901664964019, 31.010283184062946),ee.Geometry.Point(103.71700942205399, 30.47257802443212),ee.Geometry.Point(103.68974647821322, 30.475694936068486),ee.Geometry.Point(103.6548450942679, 30.472595860836336),ee.Geometry.Point(103.65496548999633, 30.473153248468016),ee.Geometry.Point(103.651349159042, 30.446982784385344),ee.Geometry.Point(103.52930802231025, 30.420937175132174),ee.Geometry.Point(103.4988523621152, 30.450113073324868),ee.Geometry.Point(103.4963240518179, 30.47491905248518),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.44297090771342, 30.503831863715725),ee.Geometry.Point(103.4362109105164, 30.473550108461772),ee.Geometry.Point(103.43450307481291, 30.42161049939123),ee.Geometry.Point(103.46724625384837, 30.446911438768485),ee.Geometry.Point(103.61455711625003, 30.36449387399765),ee.Geometry.Point(103.68620595197677, 30.358759470042916),ee.Geometry.Point(103.71908290404384, 30.3600927412579),ee.Geometry.Point(103.77645815729852, 30.389710090454876),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.87620378876306, 30.416683192727177),ee.Geometry.Point(103.87105798614739, 30.445970568446214),ee.Geometry.Point(103.68613014725887, 30.57757647693761),ee.Geometry.Point(103.65106823567564, 30.55970885901645),ee.Geometry.Point(103.62362246869168, 30.550754984101136),ee.Geometry.Point(103.59622129271824, 30.715059480617448),ee.Geometry.Point(103.599119708403, 30.71383322782776),ee.Geometry.Point(103.63093985352037, 30.691537722560533),ee.Geometry.Point(103.90764936939195, 30.73843408833962),ee.Geometry.Point(103.84272485805378, 30.733974987286174),ee.Geometry.Point(103.81973373302223, 30.766521965875278),ee.Geometry.Point(103.70528198628341, 31.036101379162403),ee.Geometry.Point(103.70293204002824, 30.93987397842904),ee.Geometry.Point(103.7489321264956, 30.855146599312512),ee.Geometry.Point(103.82210597478266, 30.875426590903576),ee.Geometry.Point(103.85075569905104, 30.906560034458742),ee.Geometry.Point(103.91618408880825, 30.901927028464208),ee.Geometry.Point(103.98263807180777, 30.952635925644),ee.Geometry.Point(103.85198195184073, 30.958967849139892),ee.Geometry.Point(103.75966964183229, 31.013065663120294),ee.Geometry.Point(103.78707081780573, 31.040948422007496),ee.Geometry.Point(103.85475997179704, 31.06174121021971),ee.Geometry.Point(103.88393586998974, 31.045719660134687),ee.Geometry.Point(103.84999765187194, 31.010109279121863),ee.Geometry.Point(103.91554197825657, 31.007839596685663),ee.Geometry.Point(103.94347824635639, 30.87328176329688),ee.Geometry.Point(103.99313925478862, 30.868800366738167),ee.Geometry.Point(104.04520817778972, 30.849577182096763),ee.Geometry.Point(104.12120909614465, 30.81684292126341),ee.Geometry.Point(103.86281310829955, 30.532307683043033),ee.Geometry.Point(103.88304404977903, 30.514056582431277),ee.Geometry.Point(104.3224527857877, 30.893811464546932)]

In [10]:
result_df = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])

for point in points:
    rTable = ee.FeatureCollection(fittedHarmonic_1.filterBounds(point).map(lambda image: ee.Feature(
        None,
        {
            'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
            'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
        }
    )))

    rTable_dict = rTable.getInfo()

    ndvi_values = [feat['properties'].get('NDVI') for feat in rTable_dict['features']]
    fitted_values = [feat['properties'].get('fitted') for feat in rTable_dict['features']]

    df = pd.DataFrame({'NDVI': ndvi_values, 'Fitted': fitted_values})

    df = df.dropna()

    if not df.empty:
        rmse = np.sqrt(mean_squared_error(df['NDVI'], df['Fitted']))
        mae = mean_absolute_error(df['NDVI'], df['Fitted'])
        r2 = r2_score(df['NDVI'], df['Fitted'])

        result_df = pd.concat([result_df, pd.DataFrame([{'RMSE': rmse, 'MAE': mae, 'R2': r2}])], ignore_index=True)

rmse = np.mean(result_df['RMSE'])
mae = np.mean(result_df['MAE'])
r2 = np.mean(result_df['R2'])
print(rmse, mae, r2)

0.18355652004713166 0.152213440452597 0.17079896219417642


In [11]:
points = [ee.Geometry.Point(103.7171610314898, 30.54972493175779),ee.Geometry.Point(103.4787774891726, 30.511684340670843),ee.Geometry.Point(103.74455328926113, 30.490164718986914),ee.Geometry.Point(103.73492609008673, 30.660636152260153),ee.Geometry.Point(103.53427100178271, 30.770722439067622),ee.Geometry.Point(103.69220790199472, 30.76936687234738),ee.Geometry.Point(103.69146769121986, 30.7438385188164),ee.Geometry.Point(103.78788237419745, 30.60895517105071),ee.Geometry.Point(103.99602875227124, 30.746968807755923),ee.Geometry.Point(104.0033996463126, 30.766624525199504),ee.Geometry.Point(103.91834675281916, 30.896687584726415),ee.Geometry.Point(103.94983246535755, 31.043904806005933),ee.Geometry.Point(103.71711644047926, 30.472925834314296),ee.Geometry.Point(103.65482279876264, 30.472685042857403),ee.Geometry.Point(103.49845550212143, 30.450358323882796),ee.Geometry.Point(103.49697062147064, 30.474923511586237),ee.Geometry.Point(103.43405270560652, 30.422092082305006),ee.Geometry.Point(103.46697870778517, 30.44713439382116),ee.Geometry.Point(103.68610339265254, 30.35813965499649),ee.Geometry.Point(103.71580992387062, 30.359718176769412),ee.Geometry.Point(103.77456303935081, 30.390454760330815),ee.Geometry.Point(103.83757459633706, 30.3949673705969),ee.Geometry.Point(103.87085286749893, 30.44555587204824),ee.Geometry.Point(103.7201976793072, 30.600277760400704),ee.Geometry.Point(103.65329778620236, 30.561519254044153),ee.Geometry.Point(103.84403583376351, 30.734322797168343),ee.Geometry.Point(103.59501733543381, 30.854821084935605),ee.Geometry.Point(103.60227229284777, 30.991452400314245),ee.Geometry.Point(103.76001745171446, 31.012842708067623),ee.Geometry.Point(103.84697884045877, 30.60314050327702),ee.Geometry.Point(103.85069773073734, 30.543665013426153)]

In [12]:
result_df = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])

for point in points:
    rTable = ee.FeatureCollection(fittedHarmonic_1.filterBounds(point).map(lambda image: ee.Feature(
        None,
        {
            'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
            'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
        }
    )))

    rTable_dict = rTable.getInfo()

    ndvi_values = [feat['properties'].get('NDVI') for feat in rTable_dict['features']]
    fitted_values = [feat['properties'].get('fitted') for feat in rTable_dict['features']]

    df = pd.DataFrame({'NDVI': ndvi_values, 'Fitted': fitted_values})

    df = df.dropna()

    if not df.empty:
        rmse = np.sqrt(mean_squared_error(df['NDVI'], df['Fitted']))
        mae = mean_absolute_error(df['NDVI'], df['Fitted'])
        r2 = r2_score(df['NDVI'], df['Fitted'])

        result_df = pd.concat([result_df, pd.DataFrame([{'RMSE': rmse, 'MAE': mae, 'R2': r2}])], ignore_index=True)

rmse = np.mean(result_df['RMSE'])
mae = np.mean(result_df['MAE'])
r2 = np.mean(result_df['R2'])
print(rmse, mae, r2)

0.14426299422454356 0.11893702256702122 0.19425609648230255


In [13]:
points = [ee.Geometry.Point(103.58009272420793, 30.399172302890303),ee.Geometry.Point(103.38048106555043, 30.347856967967243),ee.Geometry.Point(103.64897691728156, 30.34671989719861),ee.Geometry.Point(103.63306238562183, 30.344365491842392),ee.Geometry.Point(103.65286971250121, 30.422319496458734),ee.Geometry.Point(103.64021478371154, 30.42330049869049),ee.Geometry.Point(103.63662520736351, 30.41895287516338),ee.Geometry.Point(103.57232051107178, 30.48652163342625),ee.Geometry.Point(103.57890660332772, 30.485237412322856),ee.Geometry.Point(103.56894943067537, 30.4896385450626),ee.Geometry.Point(103.50341402249288, 30.536026573321603),ee.Geometry.Point(103.5121761560629, 30.541154539533068),ee.Geometry.Point(103.5068207756977, 30.539504672143295),ee.Geometry.Point(103.76563145994075, 30.603376835632858),ee.Geometry.Point(103.76370066918463, 30.598877602669923),ee.Geometry.Point(103.7579261333204, 30.596295783159974),ee.Geometry.Point(103.68813228363186, 30.71958992728776),ee.Geometry.Point(103.68804756071187, 30.71448425658156),ee.Geometry.Point(103.75724389085921, 30.575364762815095),ee.Geometry.Point(103.76006650182606, 30.57438376058334),ee.Geometry.Point(103.76700932216627, 30.573014816559944),ee.Geometry.Point(103.75418494753656, 30.570397324241565),ee.Geometry.Point(103.82964631466402, 30.44247463322031),ee.Geometry.Point(103.85162968285752, 30.454518665165665),ee.Geometry.Point(103.86080205372447, 30.478834143210104),ee.Geometry.Point(103.86157347820671, 30.482428178659195),ee.Geometry.Point(103.84026343427229, 30.56974183638671),ee.Geometry.Point(103.85112134533742, 30.537863722955628),ee.Geometry.Point(103.8680971430479, 30.513302994353243),ee.Geometry.Point(103.8868030719671, 30.43860859260698),ee.Geometry.Point(103.77969546466332, 30.38411837773386),ee.Geometry.Point(103.76675961250729, 30.385527453666754),ee.Geometry.Point(103.77242267084515, 30.383012520672608),ee.Geometry.Point(103.76159151438632, 30.81040397934224),ee.Geometry.Point(103.70163890072276, 30.947070967529303),ee.Geometry.Point(103.73197416518936, 30.94708880393351),ee.Geometry.Point(103.72080411705046, 31.011861705835866),ee.Geometry.Point(103.72257883926973, 31.015772337459737),ee.Geometry.Point(103.72988730589634, 31.00977930564391),ee.Geometry.Point(103.90308770901429, 30.968086710794193),ee.Geometry.Point(103.93012323870131, 30.95923093610205),ee.Geometry.Point(103.89799987471231, 30.86450179332265),ee.Geometry.Point(103.84756744179784, 30.805873532671935),ee.Geometry.Point(103.77079509896065, 30.707126739843375),ee.Geometry.Point(104.03504588648892, 30.293059075121445),ee.Geometry.Point(104.03863100373589, 30.290182954941976),ee.Geometry.Point(104.07643972156805, 31.001846564869826),ee.Geometry.Point(104.05241854419313, 30.97539517742079),ee.Geometry.Point(104.05665469019391, 30.973607077898357),ee.Geometry.Point(104.12643070347822, 30.87721914952707),ee.Geometry.Point(104.12993109780518, 30.8755782003394)]

In [14]:
result_df = pd.DataFrame(columns=['RMSE', 'MAE', 'R2'])

for point in points:
    rTable = ee.FeatureCollection(fittedHarmonic_1.filterBounds(point).map(lambda image: ee.Feature(
        None,
        {
            'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
            'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
        }
    )))

    rTable_dict = rTable.getInfo()

    ndvi_values = [feat['properties'].get('NDVI') for feat in rTable_dict['features']]
    fitted_values = [feat['properties'].get('fitted') for feat in rTable_dict['features']]

    df = pd.DataFrame({'NDVI': ndvi_values, 'Fitted': fitted_values})

    df = df.dropna()

    if not df.empty:
        rmse = np.sqrt(mean_squared_error(df['NDVI'], df['Fitted']))
        mae = mean_absolute_error(df['NDVI'], df['Fitted'])
        r2 = r2_score(df['NDVI'], df['Fitted'])

        result_df = pd.concat([result_df, pd.DataFrame([{'RMSE': rmse, 'MAE': mae, 'R2': r2}])], ignore_index=True)

rmse = np.mean(result_df['RMSE'])
mae = np.mean(result_df['MAE'])
r2 = np.mean(result_df['R2'])
print(rmse, mae, r2)

0.1327656872819636 0.10892416178327943 0.2609140176993579
